<img align="right" width="200" height="200" src="https://static.wikia.nocookie.net/lego/images/2/23/PPG_logo.png"> 

# <span style="color:#EF7C8E">Weekend Project - Kaggle Competition (Regression) - Machine Learning </span>
 


## <span style="color:#55BB99"> Group: Power Puff Girls</span>
### <span style="color:#726060"> Members: Wejdan Al-Ahmadi - Sara Aldubaie</span>
<hr style="border:2px solid gray"> </hr>

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

# This is for regression
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline


from sklearn.metrics import plot_confusion_matrix
from sklearn import tree

from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation



#plt.style.use("seaborn")

In [29]:
sample = pd.read_csv('./data/sample_submission.csv')
print(f"Rows: {sample.shape[0]}\nColumns: {sample.shape[1]}")
#sample.head()

Rows: 29880
Columns: 2


In [30]:
test = pd.read_csv('./data/test.csv')
print(f"Rows: {test.shape[0]}\nColumns: {test.shape[1]}")

Rows: 29880
Columns: 18


In [31]:
train = pd.read_csv('./data/train.csv')
print(f"Rows: {train.shape[0]}\nColumns: {train.shape[1]}")


Rows: 29879
Columns: 19


In [32]:
i=[2,6,7,8,9,13]
train=train.drop(train.columns[i], axis=1)

train.head()

BOROUGH             NEIGHBORHOOD TAX CLASS AT PRESENT  BLOCK  LOT  \
0        4           FLUSHING-NORTH                    1   6359    4   
1        5           NEW DORP-BEACH                    1   4067   27   
2        1  UPPER EAST SIDE (79-96)                    2   1582   46   
3        5              ELTINGVILLE                    1   5366   21   
4        2   MOTT HAVEN/PORT MORRIS                    4   2340   28   

   ZIP CODE  RESIDENTIAL UNITS  COMMERCIAL UNITS LAND SQUARE FEET  \
0     11355                  2                 0             1285   
1     10306                  2                 0             5000   
2     10028                  0                 0              -     
3     10312                  1                 0             4000   
4     10451                  0                 1            50400   

  GROSS SQUARE FEET  YEAR BUILT  SALE PRICE     Id  
0              1762        2002     1050000  57110  
1              3450        1960      750000  80619  
2               -          1958     1075000  14307  
3              1056        1960      490000  77692  
4             48952        1958    11000000  21755

In [33]:
# strip white spaces
train['LAND SQUARE FEET'] = train['LAND SQUARE FEET'].str.strip()
train['GROSS SQUARE FEET'] = train['GROSS SQUARE FEET'].str.strip()
# replace with naans
train['LAND SQUARE FEET']=train['LAND SQUARE FEET'].replace({'-':np.nan,'0':np.nan})
train['GROSS SQUARE FEET']=train['GROSS SQUARE FEET'].replace({'-':np.nan,'0':np.nan})
train['YEAR BUILT']=train['YEAR BUILT'].replace({0:np.nan})

In [34]:
# fill null value using forward filling method in YEAR BUILT column
train.update(train['YEAR BUILT'].fillna(method="ffill"))

# convert column type to int
train['YEAR BUILT'] = train['YEAR BUILT'].astype(int)


In [35]:
# convert column type to float
train['LAND SQUARE FEET'] = train['LAND SQUARE FEET'].astype(float)
train['GROSS SQUARE FEET'] = train['GROSS SQUARE FEET'].astype(float)

In [36]:
# fill column based on YEAR BUILT column 
train['LAND SQUARE FEET']= train['LAND SQUARE FEET'].fillna(round(train.groupby('YEAR BUILT')['LAND SQUARE FEET'].transform('mean')))
train['GROSS SQUARE FEET']= train['GROSS SQUARE FEET'].fillna(round(train.groupby('YEAR BUILT')['GROSS SQUARE FEET'].transform('mean')))

# fill null value using forward filling method in YEAR BUILT column
train.update(train['LAND SQUARE FEET'].fillna(method="ffill"))
train.update(train['GROSS SQUARE FEET'].fillna(method="ffill"))

# convert column type to int
train['GROSS SQUARE FEET'] = train['GROSS SQUARE FEET'].astype(int)
train['LAND SQUARE FEET'] = train['LAND SQUARE FEET'].astype(int)

In [37]:
# dummies 
#encoding using dummies 
train = pd.get_dummies(train, columns = ['TAX CLASS AT PRESENT','NEIGHBORHOOD'])


In [38]:
#split data into new train and val 
train, val = train_test_split(
    train,
    test_size=0.3,     #test -----> 30%
    train_size = 0.7,  #train ----> 70%
    random_state=42
)

X_train = train.drop(columns = 'SALE PRICE')
y_train = train['SALE PRICE']


X_val = val.drop(columns = 'SALE PRICE')
y_val = val['SALE PRICE']

print(X_train.shape)
print(X_val.shape)

(20915, 271)
(8964, 271)


In [39]:
X_train.head()

BOROUGH  BLOCK   LOT  ZIP CODE  RESIDENTIAL UNITS  COMMERCIAL UNITS  \
11335        1    915    38     10016                  0                 0   
14081        3   1850    44     11216                  3                 1   
17597        1    705    45     10001                  0                 1   
13776        4   8789    49     11001                  1                 0   
11124        1   1049  1026     10019                  1                 0   

       LAND SQUARE FEET  GROSS SQUARE FEET  YEAR BUILT     Id  ...  \
11335              3994               2055        1955  10129  ...   
14081              1600               4000        1910  27636  ...   
17597              9875              11250        1900   6312  ...   
13776              1800               1188        1960  55952  ...   
11124              3994               2055        1955   9496  ...   

       NEIGHBORHOOD_WILLIAMSBURG-EAST  NEIGHBORHOOD_WILLIAMSBURG-NORTH  \
11335                               0                                0   
14081                               0                                0   
17597                               0                                0   
13776                               0                                0   
11124                               0                                0   

       NEIGHBORHOOD_WILLIAMSBURG-SOUTH  NEIGHBORHOOD_WILLOWBROOK  \
11335                                0                         0   
14081                                0                         0   
17597                                0                         0   
13776                                0                         0   
11124                                0                         0   

       NEIGHBORHOOD_WINDSOR TERRACE  NEIGHBORHOOD_WOODHAVEN  \
11335                             0                       0   
14081                             0                       0   
17597                             0                       0   
13776                             0                       0   
11124                             0                       0   

       NEIGHBORHOOD_WOODLAWN  NEIGHBORHOOD_WOODROW  NEIGHBORHOOD_WOODSIDE  \
11335                      0                     0                      0   
14081                      0                     0                      0   
17597                      0                     0                      0   
13776                      0                     0                      0   
11124                      0                     0                      0   

       NEIGHBORHOOD_WYCKOFF HEIGHTS  
11335                             0  
14081                             0  
17597                             0  
13776                             0  
11124                             0  

[5 rows x 271 columns]

In [40]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20915 entries, 11335 to 23654
Columns: 271 entries, BOROUGH to NEIGHBORHOOD_WYCKOFF HEIGHTS
dtypes: int32(3), int64(7), uint8(261)
memory usage: 6.7 MB


In [41]:
def cleaner(df):
    "Return Only Numeric Features"
    
    # Create boolean array with only columns that are float64
    columns_bool = (df.dtypes != "category").values & (df.dtypes != "object").values 

    # Return only numeric columns
    df = df.iloc[:, columns_bool].dropna()

    return df


In [42]:
for col in X_train.columns: 
    print(col)

BOROUGH
BLOCK
LOT
ZIP CODE
RESIDENTIAL UNITS
COMMERCIAL UNITS
LAND SQUARE FEET
GROSS SQUARE FEET
YEAR BUILT
Id
TAX CLASS AT PRESENT_ 
TAX CLASS AT PRESENT_1
TAX CLASS AT PRESENT_1A
TAX CLASS AT PRESENT_1B
TAX CLASS AT PRESENT_1C
TAX CLASS AT PRESENT_2
TAX CLASS AT PRESENT_2A
TAX CLASS AT PRESENT_2B
TAX CLASS AT PRESENT_2C
TAX CLASS AT PRESENT_4
NEIGHBORHOOD_AIRPORT LA GUARDIA
NEIGHBORHOOD_ALPHABET CITY
NEIGHBORHOOD_ANNADALE
NEIGHBORHOOD_ARDEN HEIGHTS
NEIGHBORHOOD_ARROCHAR
NEIGHBORHOOD_ARROCHAR-SHORE ACRES
NEIGHBORHOOD_ARVERNE
NEIGHBORHOOD_ASTORIA
NEIGHBORHOOD_BATH BEACH
NEIGHBORHOOD_BATHGATE
NEIGHBORHOOD_BAY RIDGE
NEIGHBORHOOD_BAYCHESTER
NEIGHBORHOOD_BAYSIDE
NEIGHBORHOOD_BEDFORD PARK/NORWOOD
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_BEECHHURST
NEIGHBORHOOD_BELLE HARBOR
NEIGHBORHOOD_BELLEROSE
NEIGHBORHOOD_BELMONT
NEIGHBORHOOD_BENSONHURST
NEIGHBORHOOD_BERGEN BEACH
NEIGHBORHOOD_BLOOMFIELD
NEIGHBORHOOD_BOERUM HILL
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_BRIARWOOD
NEIGHBORHOOD_BRIGHTON BE

In [60]:
for col in test.columns: 
    print(col)

BOROUGH
BLOCK
LOT
ZIP CODE
RESIDENTIAL UNITS
COMMERCIAL UNITS
LAND SQUARE FEET
GROSS SQUARE FEET
YEAR BUILT
Id
TAX CLASS AT PRESENT_ 
TAX CLASS AT PRESENT_1
TAX CLASS AT PRESENT_1A
TAX CLASS AT PRESENT_1B
TAX CLASS AT PRESENT_1C
TAX CLASS AT PRESENT_2
TAX CLASS AT PRESENT_2A
TAX CLASS AT PRESENT_2B
TAX CLASS AT PRESENT_2C
TAX CLASS AT PRESENT_4
NEIGHBORHOOD_AIRPORT LA GUARDIA
NEIGHBORHOOD_ALPHABET CITY
NEIGHBORHOOD_ANNADALE
NEIGHBORHOOD_ARDEN HEIGHTS
NEIGHBORHOOD_ARROCHAR
NEIGHBORHOOD_ARROCHAR-SHORE ACRES
NEIGHBORHOOD_ARVERNE
NEIGHBORHOOD_ASTORIA
NEIGHBORHOOD_BATH BEACH
NEIGHBORHOOD_BATHGATE
NEIGHBORHOOD_BAY RIDGE
NEIGHBORHOOD_BAYCHESTER
NEIGHBORHOOD_BAYSIDE
NEIGHBORHOOD_BEDFORD PARK/NORWOOD
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_BEECHHURST
NEIGHBORHOOD_BELLE HARBOR
NEIGHBORHOOD_BELLEROSE
NEIGHBORHOOD_BELMONT
NEIGHBORHOOD_BENSONHURST
NEIGHBORHOOD_BERGEN BEACH
NEIGHBORHOOD_BLOOMFIELD
NEIGHBORHOOD_BOERUM HILL
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_BRIARWOOD
NEIGHBORHOOD_BRIGHTON BE

In [61]:
test.shape

(29880, 269)

In [71]:
X_val .shape

(8964, 269)

In [72]:
X_train .shape

(20915, 269)

In [45]:
#train_numeric = cleaner(train)
#val_numeric = cleaner(val)

In [70]:
# drop columns to make it fit the test data
X_val=X_val.drop(['NEIGHBORHOOD_BRONX PARK', 'NEIGHBORHOOD_CO-OP CITY'], axis=1)
X_train=X_train.drop(['NEIGHBORHOOD_BRONX PARK', 'NEIGHBORHOOD_CO-OP CITY'], axis=1)

In [73]:
reg_forest = RandomForestRegressor(n_estimators = 5, random_state = 0, criterion = 'mse')
reg_forest.fit(X_train, y_train)

preds_forest = reg_forest.predict(X_val)
mean_absolute_error(y_true=y_val, y_pred=preds_forest)

878348.5864569388

In [47]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29880 entries, 0 to 29879
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   BOROUGH                    29880 non-null  int64 
 1   NEIGHBORHOOD               29880 non-null  object
 2   BUILDING CLASS CATEGORY    29880 non-null  object
 3   TAX CLASS AT PRESENT       29880 non-null  object
 4   BLOCK                      29880 non-null  int64 
 5   LOT                        29880 non-null  int64 
 6   EASE-MENT                  29880 non-null  object
 7   BUILDING CLASS AT PRESENT  29880 non-null  object
 8   ADDRESS                    29880 non-null  object
 9   APARTMENT NUMBER           29880 non-null  object
 10  ZIP CODE                   29880 non-null  int64 
 11  RESIDENTIAL UNITS          29880 non-null  int64 
 12  COMMERCIAL UNITS           29880 non-null  int64 
 13  TOTAL UNITS                29880 non-null  int64 
 14  LAND S

In [48]:
# cleaning test data
i=[2,6,7,8,9,13]
test=test.drop(test.columns[i], axis=1)

In [49]:
# strip white spaces
test['LAND SQUARE FEET'] = test['LAND SQUARE FEET'].str.strip()
test['GROSS SQUARE FEET'] = test['GROSS SQUARE FEET'].str.strip()
# replace with naans
test['LAND SQUARE FEET']=test['LAND SQUARE FEET'].replace({'-':np.nan,'0':np.nan})
test['GROSS SQUARE FEET']=test['GROSS SQUARE FEET'].replace({'-':np.nan,'0':np.nan})
test['YEAR BUILT']=test['YEAR BUILT'].replace({0:np.nan})

In [53]:
# fill null value using forward filling method in YEAR BUILT column
test.update(test['YEAR BUILT'].fillna(method="bfill"))

# convert column type to int
test['YEAR BUILT'] = test['YEAR BUILT'].astype(int)

In [56]:
# convert column type to float
train['LAND SQUARE FEET'] = train['LAND SQUARE FEET'].astype(float)
train['GROSS SQUARE FEET'] = train['GROSS SQUARE FEET'].astype(float)

In [57]:
# fill column based on YEAR BUILT column 
test['LAND SQUARE FEET']= test['LAND SQUARE FEET'].fillna(round(test.groupby('YEAR BUILT')['LAND SQUARE FEET'].transform('mean')))
test['GROSS SQUARE FEET']= test['GROSS SQUARE FEET'].fillna(round(test.groupby('YEAR BUILT')['GROSS SQUARE FEET'].transform('mean')))

# fill null value using forward filling method in YEAR BUILT column
test.update(test['LAND SQUARE FEET'].fillna(method="ffill"))
test.update(test['GROSS SQUARE FEET'].fillna(method="ffill"))

# convert column type to int
test['GROSS SQUARE FEET'] = test['GROSS SQUARE FEET'].astype(int)
test['LAND SQUARE FEET'] = test['LAND SQUARE FEET'].astype(int)

In [58]:
# dummies 
#encoding using dummies 
test = pd.get_dummies(test, columns = ['TAX CLASS AT PRESENT','NEIGHBORHOOD'])

In [65]:
test.columns.difference(X_val.columns)

Index(['NEIGHBORHOOD_CITY ISLAND-PELHAM STRIP', 'NEIGHBORHOOD_EAST RIVER',
       'NEIGHBORHOOD_PELHAM BAY'],
      dtype='object')

In [64]:
X_val.columns.difference(test.columns)

Index(['NEIGHBORHOOD_BRONX PARK', 'NEIGHBORHOOD_CO-OP CITY',
       'NEIGHBORHOOD_FRESH KILLS', 'NEIGHBORHOOD_ROSSVILLE-PORT MOBIL',
       'NEIGHBORHOOD_VAN CORTLANDT PARK'],
      dtype='object')

In [74]:
preds_test = reg_forest.predict(test)

In [77]:
test['SALE PRICE'] = preds_test

In [78]:
results = test[['Id','SALE PRICE']]

Id  SALE PRICE
0  16203    962126.2
1  24057    148258.2
2  13182  10898827.4
3  66523   1294919.6
4  32060    570645.8

In [80]:
# saving the dataframe
results.to_csv('results.csv',  index=False)